In [29]:
# ! pip install openai
# ! pip install google
# ! pip install llama-index-readers-web
# ! pip install llama-index-llms-openai
# %pip install llama-index-program-openai
# %pip install llama-index-llms-llama-api
! pip install llama-index-embeddings-openai

In [5]:
import os
from openai import OpenAI as open_ai
import re
os.environ["OPENAI_API_KEY"] = 'apikey'
client = open_ai()

In [6]:
setup_string = """
Your job is to return Google search queries, specifically Google Dorks, to find more
about a subject given a dumped text file of all of the information we currently know
about the subject. Not all of this information should be used in every query.
You can output up to twenty queries, and they should be formatted in a bullet point list
format, where every bullet point contains only the Google Dork search query and nothing
else (without quotes).

The end goal is to create an OSINT (open source intelligence) report about
the given subject. For example, if the user wants to find more about a person,
you would help them find out about their location, age, education, associated
institutions, relevant connections, and so forth.
""" # add osint flowchart

In [7]:
prompt = """
I want to find out more about the person Jai Sharma using Google Dorks. Here are
some things I know about him:
- Went to Monta Vista High School
- Is friends with Milind Maiti
- Might be associated with Berkeley
- Research oriented and possibly interested in Machine Learning, wrote a paper with Christopher Sun
"""

In [16]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": setup_string},
    {"role": "user", "content": prompt}
  ]
)

result = completion.choices[0].message.content
search_list = result.split("\n")
search_list = [i[2:] for i in search_list]
print(result)
print(search_list)

- "Jai Sharma" "Monta Vista High School" "Berkeley"
- "Jai Sharma" "Milind Maiti"
- "Jai Sharma" "Machine Learning" "Christopher Sun"
- "Jai Sharma" "research" "Berkeley"
- "Jai Sharma" "publication" "Christopher Sun"
- "Jai Sharma" "Machine Learning" site:berkeley.edu
- "Jai Sharma" "Monta Vista High School" "alumni"
- "Jai Sharma" "research group" "Berkeley"
- "Jai Sharma" "Christopher Sun" "co-author"
- "Jai Sharma" "Machine Learning" "publication" -site:github.com
- "Jai Sharma" "research interests" "Berkeley"
- "Jai Sharma" "Monta Vista High School" "facebook"
['"Jai Sharma" "Monta Vista High School" "Berkeley"', '"Jai Sharma" "Milind Maiti"', '"Jai Sharma" "Machine Learning" "Christopher Sun"', '"Jai Sharma" "research" "Berkeley"', '"Jai Sharma" "publication" "Christopher Sun"', '"Jai Sharma" "Machine Learning" site:berkeley.edu', '"Jai Sharma" "Monta Vista High School" "alumni"', '"Jai Sharma" "research group" "Berkeley"', '"Jai Sharma" "Christopher Sun" "co-author"', '"Jai Shar

In [17]:
from googlesearch import search

# query = "Geeksforgeeks"
urls = []

for query in search_list:
  for j in search(query, tld="co.in", num=10, stop=10, pause=2):
      urls.append(j)
      print(j)

In [18]:
urls

['https://www.linkedin.com/in/rithwik-nukala',
 'https://www.academia.edu/103814896/Exploring_Algebraic_Conditions_on_Counterexamples_of_the_Collatz_Conjecture',
 'https://www.linkedin.com/in/nithyaappannagaari',
 'https://www.instagram.com/mvresearchclub/',
 'https://ieeexplore.ieee.org/iel7/9987717/9987755/09988121.pdf',
 'https://arxiv.org/abs/2108.06628',
 'https://www.researchgate.net/scientific-contributions/Milind-Maiti-2199726740',
 'https://vixra.org/author/jai_sharma',
 'https://vixra.org/author/milind_maiti',
 'https://arxiv.org/abs/2205.11230',
 'https://www.researchgate.net/scientific-contributions/Jai-Sharma-2199738343',
 'https://www.preprints.org/manuscript/202207.0097/v1',
 'https://www.researchsquare.com/article/rs-1769921/v1.pdf',
 'https://www.researcher-app.com/paper/7041375',
 'https://arxiv.org/abs/2108.06628',
 'https://www.researchgate.net/scientific-contributions/Jai-Sharma-2199738343',
 'https://www.researchgate.net/publication/363724723_Leveraging_Machine_Le

In [27]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import SummaryIndex, Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.llms.llama_api import LlamaAPI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# urls = ["https://en.wikipedia.org/wiki/Eminem"]
documents = SimpleWebPageReader().load_data(urls[:5])

for i in range(len(documents)):
  documents[i].text = re.sub(r'\\s+', ' ', documents[i].text)
  documents[i].text = re.sub(r' +', ' ', documents[i].text)
  documents[i].text = re.sub(r'\\n', '\n', documents[i].text)
  documents[i].text = re.sub(r'\n +', '\n', documents[i].text)
  documents[i].text = re.sub(r'\n+', '\n', documents[i].text)
  print(documents[i].text)

parser = HTMLNodeParser(
    # tags=["p", "h1"],
    chunk_lines=100,
    chunk_lines_overlap=50,
    max_chars=1500,
    )

nodes = parser.get_nodes_from_documents(documents)
nodes = [node for node in nodes if len(node.text) > 0]
for node in nodes:
  print(node.text)
  print(node.metadata)

<html><head>
<script type="text/javascript">
window.onload = function() {
// Parse the tracking code from cookies.
var trk = "bf";
var trkInfo = "bf";
var cookies = document.cookie.split("; ");
for (var i = 0; i < cookies.length; ++i) {
if ((cookies[i].indexOf("trkCode=") == 0) && (cookies[i].length > 8)) {
trk = cookies[i].substring(8);
}
else if ((cookies[i].indexOf("trkInfo=") == 0) && (cookies[i].length > 8)) {
trkInfo = cookies[i].substring(8);
}
}
if (window.location.protocol == "http:") {
// If "sl" cookie is set, redirect to https.
for (var i = 0; i < cookies.length; ++i) {
if ((cookies[i].indexOf("sl=") == 0) && (cookies[i].length > 3)) {
window.location.href = "https:" + window.location.href.substring(window.location.protocol.length);
return;
}
}
}
// Get the new domain. For international domains such as
// fr.linkedin.com, we convert it to www.linkedin.com
// treat .cn similar to .com here
var domain = location.host;
if (domain != "www.linkedin.com" && domain != "www.linkedi

In [22]:
# print(nodes[2].text)

In [23]:
len(nodes)

12

In [30]:
index = VectorStoreIndex(nodes=nodes)

In [25]:
# Settings.llm = client # OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [26]:
# query_engine = index.as_query_engine()
# response = query_engine.query(prompt)
# print(response)

Jai Sharma may have attended Monta Vista High School, is acquainted with Milind Maiti, could have ties to Berkeley, and is likely research-oriented with a potential interest in Machine Learning, having collaborated on a paper with Christopher Sun.


In [36]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# # configure response synthesizer
# response_synthesizer = get_response_synthesizer()

# # assemble query engine
# query_engine = RetrieverQueryEngine(
#     retriever=retriever,
#     response_synthesizer=response_synthesizer,
#     node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
# )

# # query
# response = query_engine.query(prompt)
# print(response)

relevant_nodes = retriever.retrieve(prompt)

In [34]:
relevant_nodes

[NodeWithScore(node=TextNode(id_='cb71d58e-1ac2-4250-95e4-5e58823d9534', embedding=None, metadata={'tag': 'h3'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://www.academia.edu/103814896/Exploring_Algebraic_Conditions_on_Counterexamples_of_the_Collatz_Conjecture', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='dad23f3b3963addeeda37cc66be1492d32af86d45254b74ca19920ebb7c67fdd'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d36e4e10-ff9c-4fa0-bca5-d4868d6711a3', node_type=<ObjectType.TEXT: '1'>, metadata={'tag': 'p'}, hash='289e985a77bb3e80d7323c00eafc6fbbcbda9cf864eeca09062b0c9f285accbd'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6f873159-bbb8-4e34-8fc8-177ce22ba465', node_type=<ObjectType.TEXT: '1'>, metadata={'tag': 'p'}, hash='fe3dd9a1a385011b2cdebb4b5b7e9b333c5c6dd3f80bbd31e03c83f9c8075f01')}, text='RELATED PAPERS', mimetype='text/plain', start_char_i